# Prédictions avec scikitlearn

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
from serde import serialize, deserialize
from serde.json import to_json, from_json
import pandas as pd
from requests import get 
from dataclasses import dataclass
from pays import Countries

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from rich import print
from rich.table import Table

import sklearn.metrics

On commence par charger le dataframe.

In [4]:
df = pd.read_pickle("my_df.pkl")

In [5]:
df

,description,prix,lieux,duree,diff,theme
0,plaisirs neige &#224; sixt-fer-&#224;-cheval,690,metropole,8,Initié,multi-activités
1,haute clar&#233;e &quot;espace trappeur&quot;,860,metropole,7,Découverte,multi-activités
2,aventures hivernales sur le plateau de l&#39;a...,720,metropole,6,Découverte,multi-activités
3,neige et soleil du queyras en famille,730,metropole,7,Découverte,multi-activités
4,plaisirs neige dans les aravis,830,metropole,7,Découverte,multi-activités
...,...,...,...,...,...,...
383,plongée à la réunion,1540,dest_éloignée,10,Initié,multi-activités
384,randonnée dans le parc naturel de topes de col...,1690,dest_éloignée,14,Découverte,rando nature
385,"randonnée jusqu'au mont ryten, vue sur kvalvika",1930,dest_éloignée,15,Découverte,rando nature
386,drapée masai flottant dans les terres du rift,2350,dest_éloignée,8,Découverte,rando nature


In [6]:
df.shape

(388, 6)

On définit la **variable à expliquer** et les **variables explicatives**.

In [7]:
X = df.drop(["description", "prix"], axis = 1)
y = df["prix"]

On sépare notre échantillons en deux, une partie pour l'apprentissage et une partie pour les test. 

In [8]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

On transforme les variables catégorielles lors de la phase de preprocessing à l'aide d'un `one hot encoder`.

On pourrait utiliser d'autres méthodes d'encodage (code en commentaire ci-dessous)

In [2]:
# from sklearn.preprocessing import FunctionTransformer

# encoding = { "low": 0, "med": 1, "high": 2, "vhigh": 3 }

# def grad_encoder(df):
#   for col in df.columns:
#     df[col] = df[col].apply(lambda x: encoding[x])
#   return df

# eval_encoder = Pipeline(
#   steps=[
#     ('grad', FunctionTransformer(grad_encoder))
#   ]
# )

# def num_encoder(df):
#   for col in df.columns:
#     df[col] = df[col].apply(lambda x: 4 if x == "4more" else x)
#     df[col] = df[col].apply(lambda x: 4 if x == "more" else x)
#   return df

# num_encoder = Pipeline(
#   steps=[
#     ('num', FunctionTransformer(num_encoder))
#   ]
# )

In [10]:
one_hot_encoder = Pipeline(
  steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
  ]
)

In [11]:
preprocessor = ColumnTransformer(
  transformers=[
    ('categorical', one_hot_encoder, ['lieux', 'diff', 'theme']),
  ]
)

Un premier test d'apprentissage sans *tuning* des paramètres donne des résultats très décevants...

In [12]:
models_names = ["Random Forest", "Linear Regression", "ElasticNet"]

pipelines = [
  Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestRegressor())]),
  Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LinearRegression())]),
  Pipeline(steps=[('preprocessor', preprocessor), ('classifier', ElasticNet())])
]

In [13]:
for p, name in zip(pipelines, models_names):
  p.fit(X_tr, y_tr)
  y_pred = p.predict(X_te)
  print("Score ({}) : {:2.1f}".format(
    name,
    p.score(X_te, y_te)
  ))


Score (Random Forest) : 0.5

Score (Linear Regression) : 0.4

Score (ElasticNet) : 0.2

On va donc essayer d'optimiser les âramètres d'apprentissages. La liste dans laquelle on va stocker kes résultats.

In [14]:
results = dict()

## reg lin pure 

In [15]:
p = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('reg', LinearRegression())
  ]
)

p.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('one_hot',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    ['lieux', 'diff', 'theme'])])),
  ('reg', LinearRegression())],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('categorical',
                                  Pipeline(steps=[('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['lieux', 'diff', 'theme'])]),
 'reg': LinearRegression(),
 'preprocessor__n_jobs': None,
 'preprocessor__remainder': 'drop',
 'preprocessor__sparse_threshold': 0.3,
 'preprocessor__transformer_weights': None,
 'preprocessor__transformers': [('categorical',
   Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))]),
   ['lieux', 'diff', 'theme'])],
 'pre

In [16]:
g = GridSearchCV(
    p,
    {
        "reg__fit_intercept": [True, False]
    },
   
)
g.fit(X_tr, y_tr)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('one_hot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['lieux',
                                                                          'diff',
                                                                          'theme'])])),
                                       ('reg', LinearRegression())]),
             param_grid={'reg__fit_intercept': [True, False]})

In [17]:
results["Lineaire pure"] = g

## rf regressor

In [18]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg", RandomForestRegressor())
    ]
)
p.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('one_hot',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    ['lieux', 'diff', 'theme'])])),
  ('Reg', RandomForestRegressor())],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('categorical',
                                  Pipeline(steps=[('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['lieux', 'diff', 'theme'])]),
 'Reg': RandomForestRegressor(),
 'preprocessor__n_jobs': None,
 'preprocessor__remainder': 'drop',
 'preprocessor__sparse_threshold': 0.3,
 'preprocessor__transformer_weights': None,
 'preprocessor__transformers': [('categorical',
   Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))]),
   ['lieux', 'diff', 'theme'

In [19]:
g = GridSearchCV(
    p,
    {
        "Reg__n_estimators": range(50, 300, 20),
    },
    n_jobs=-1,
   
)
g.fit(X_tr, y_tr)


GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('one_hot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['lieux',
                                                                          'diff',
                                                                          'theme'])])),
                                       ('Reg', RandomForestRegressor())]),
             n_jobs=-1, param_grid={'Reg__n_estimators': range(50, 300, 20)})

In [20]:
results["random forest"] = g

## reseau de neurone

In [21]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg", MLPRegressor())
    ]
)
p.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('one_hot',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    ['lieux', 'diff', 'theme'])])),
  ('Reg', MLPRegressor())],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('categorical',
                                  Pipeline(steps=[('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['lieux', 'diff', 'theme'])]),
 'Reg': MLPRegressor(),
 'preprocessor__n_jobs': None,
 'preprocessor__remainder': 'drop',
 'preprocessor__sparse_threshold': 0.3,
 'preprocessor__transformer_weights': None,
 'preprocessor__transformers': [('categorical',
   Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))]),
   ['lieux', 'diff', 'theme'])],
 'preprocesso

In [22]:
g = GridSearchCV(
    p,
    {
        'Reg__hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,),],
        "Reg__max_iter": [5000],
        "Reg__activation": ["relu", "logistic"],
    },
    n_jobs=-1,
)
g.fit(X_tr, y_tr)

C:\Users\arkan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('one_hot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['lieux',
                                                                          'diff',
                                                                          'theme'])])),
                                       ('Reg', MLPRegressor())]),
             n_jobs=-1,
             param_grid={'Reg__activation': ['relu', 'logistic'],
                         'Reg__hidden_layer_sizes': [(50,), (100,), (150,),
                                                     (200,), (250,), (300,)],
                         'Reg__max_iter': [5000]})

In [23]:
results["reseau neurone"]=g

## SVM

In [24]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg", SVR())
    ]
)
p.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('one_hot',
                                                     OneHotEncoder(handle_unknown='ignore'))]),
                                    ['lieux', 'diff', 'theme'])])),
  ('Reg', SVR())],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('categorical',
                                  Pipeline(steps=[('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                  ['lieux', 'diff', 'theme'])]),
 'Reg': SVR(),
 'preprocessor__n_jobs': None,
 'preprocessor__remainder': 'drop',
 'preprocessor__sparse_threshold': 0.3,
 'preprocessor__transformer_weights': None,
 'preprocessor__transformers': [('categorical',
   Pipeline(steps=[('one_hot', OneHotEncoder(handle_unknown='ignore'))]),
   ['lieux', 'diff', 'theme'])],
 'preprocessor__verbose': False

In [25]:
g = GridSearchCV(
    p,
    {
        "Reg__C": [ 0.1, 1., 10., 100,],
        "Reg__epsilon": [0.01, 0.1, 1., 10.]
    },
    n_jobs=-1,
)
g.fit(X_tr, y_tr)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('one_hot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['lieux',
                                                                          'diff',
                                                                          'theme'])])),
                                       ('Reg', SVR())]),
             n_jobs=-1,
             param_grid={'Reg__C': [0.1, 1.0, 10.0, 100],
                         'Reg__epsilon': [0.01, 0.1, 1.0, 10.0]})

In [26]:
results["SVR"] = g

## Bilan 

In [27]:
tbl = Table(
    title="Résumé des résultats de crossvalidation.",
    show_header=True,
)
tbl.add_column("Nom")
tbl.add_column("Score Cross validation")
tbl.add_column("Score entrainement")
tbl.add_column("Choix Hyperparamètres")
for nom, modele in results.items():
    tbl.add_row(
        nom, 
        f"{modele.best_score_:.2f}", 
        f"{modele.score(X_tr, y_tr):.2f}",
        str(modele.best_params_),
    )
    
print(tbl)

                          Résumé des résultats de crossvalidation.                           
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Nom            ┃ Score Cross validation ┃ Score entrainement ┃ Choix Hyperparamètres      ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Lineaire pure  │ 0.19                   │ 0.37               │ {'reg__fit_intercept':     │
│                │                        │                    │ False}                     │
│ random forest  │ 0.21                   │ 0.42               │ {'Reg__n_estimators': 190} │
│ reseau neurone │ 0.25                   │ 0.34               │ {'Reg__activation':        │
│                │                        │                    │ 'logistic',                │
│                │                        │                    │ 'Reg__hidden_layer_sizes': │
│                │                        │                    │ (200,), 'Reg__max_iter':   │
│                │                        │                    │ 5000}                      │
│ SVR            │ 0.14                   │ 0.25               │ {'Reg__C': 100,            │
│                │                        │                    │ 'Reg__epsilon': 10.0}      │
└────────────────┴────────────────────────┴────────────────────┴────────────────────────────┘

Les résultats sont très mauvais, aucn score ne dépasse *O.5*.

Plusieurs pistes : 
- Echantillon trop petit (388 séjours). Il en faudrait au minimum 1000.
- trop de pertes d'information lors du regroupement des modalités.
- Phase d'optimisation des paramètres très mal configurées.

L'objectif était de prévoir le prix pour certains types de séjours donnés (séjours que nous voulons développer dans le cadre de notre création d'entreprise). Une fois les prix du marchés déterminés, nous aurions fait une phase d'optimisation pour savoir quel est la facturation minimum que nous aurions pu mettre en place en fonction de nos couts et des prix du marché. Nous n'avons pas développé cette phaase car le résultat de la phase d'apprentissage est trop mauvais. 

Il faut d'abord avoir de meilleurs résultats. Nous pourrions peut être abandonné cette troisième phase et nous contenter d'essayer d'anticiper les prix des séjours présentés sur les sites qui n'ont pas encore d'information sur leurs prix.

## Méthode à suivre

- découpage échantillon en (train/test)
- apprentissage sur échantillon entrainement pour tous les modèles
- comparaison des modèles par cross validation sur echantillon d'entrainement (si pas de CV, découpage échantillon en 3 : train/valid/test)
- predict sur échantillon test (garde fou). 
    - Si surapprentissage (score CV supérieur à score test), reprendre tout à zéro. Pb peut venir d'un découpage malchanceux, d'une CV malchanceuse...
    - Si score test ok, on réentraine sur l'échantillon complet. 